In [1]:
import logging

# Cria um logger com o nome 'jupyter_logger'
logger = logging.getLogger('jupyter_logger')

# Define o nível de log para DEBUG, então todas as mensagens de log serão mostradas
logger.setLevel(logging.DEBUG)

# Cria um manipulador de log que escreve as mensagens de log na saída padrão
handler = logging.StreamHandler()

# Define o nível de log do manipulador para DEBUG
handler.setLevel(logging.DEBUG)

# Cria um formatador de log que adiciona a data e hora à mensagem de log
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Adiciona o formatador ao manipulador
handler.setFormatter(formatter)

# Adiciona o manipulador ao logger
logger.addHandler(handler)

In [2]:
from minio import Minio
from minio.error import S3Error
import os

def minion_upload(source_file,destination_file,bucket_name="raw"):

    client = Minio(endpoint="minio:9000", 
                   access_key="wFw2nWmmNHNivuqh", 
                   secret_key="Y09TobUR4MGJTuhphCnMMgmdoIBfn1gs", 
                   secure=False)

    # Make the bucket if it doesn't exist.
    found = client.bucket_exists(bucket_name)
    if not found:
        client.make_bucket(bucket_name)
        logger.debug(f"Created bucket {bucket_name}")
    else:
        logger.debug(f"Bucket {bucket_name} already exists")

    # Upload the file, renaming it in the process
    client.fput_object(
        bucket_name, destination_file, source_file,
    )
    logger.debug(
        f"{source_file} successfully uploaded as object {destination_file} to bucket {bucket_name}"
    )
    os.remove(source_file)

In [3]:
import json

def writing_json(data):
    data_obj = json.loads(data)
    path = f'user/{data_obj["uid"]}.json'

    # Abra o arquivo onde você deseja escrever
    with open(path, 'w') as f:
        # Use json.dump para escrever o objeto no arquivo
        json.dump(data_obj, f)
    return path

In [4]:
from confluent_kafka import Consumer
from time import sleep

class ExampleConsumer:
    broker = "kafka-broker:9092"
    topic = "user"
    group_id = "python"

    def start_listener(self):
        consumer_config = {
            'bootstrap.servers': self.broker,
            'group.id': self.group_id,
            'auto.offset.reset': 'latest',
            'enable.auto.commit': 'false',
            'max.poll.interval.ms': '86400000'
        }

        consumer = Consumer(consumer_config)
        consumer.subscribe([self.topic])

        try:
            while True:

                msg = consumer.poll(0)
                
                if msg is None:
                    sleep(5)
                    continue
                if msg.error():
                    print("Error reading message : {}".format(msg.error()))
                    continue
                # You can parse message and save to data base here
                logger.debug('----------------------------')
                logger.debug("partition: {} \n offset: {} \n topic: {} \n timestamp: {} \n value: {}".format(msg.partition(), msg.offset(),msg.topic(),msg.timestamp(),msg.value()))
                #sua regra
                
                # print()
                path=writing_json(msg.value().decode("utf-8"))
                minion_upload(source_file=path,destination_file=path)
                                
                consumer.commit()

        except Exception as ex:
            print("Kafka Exception : {}", ex)

        finally:
            print("closing consumer")
            consumer.close()

In [5]:
my_consumer = ExampleConsumer()
my_consumer.start_listener()

2024-05-20 00:00:37,072 - jupyter_logger - DEBUG - ----------------------------
2024-05-20 00:00:37,074 - jupyter_logger - DEBUG - partition: 0 
 offset: 1599 
 topic: user 
 timestamp: (1, 1716163134543) 
 value: b'{"id": 4186, "uid": "72263790-47ef-4d78-8a30-557fc3d30398", "password": "NSYho1reP2", "first_name": "Lawrence", "last_name": "Stamm", "username": "lawrence.stamm", "email": "lawrence.stamm@email.com", "avatar": "https://robohash.org/corruptinobisfuga.png?size=300x300&set=set1", "gender": "Non-binary", "phone_number": "+231 207-248-6533 x90187", "social_insurance_number": "963859806", "date_of_birth": "2000-01-07", "employment": {"title": "Human Associate", "key_skill": "Confidence"}, "address": {"city": "Port Zacharychester", "street_name": "Von Drive", "street_address": "527 Chong Ranch", "zip_code": "90227", "state": "Oklahoma", "country": "United States", "coordinates": {"lat": 8.939408050587232, "lng": 21.856833973526363}}, "credit_card": {"cc_number": "6771-8975-1363-3

closing consumer


KeyboardInterrupt: 